## Expansão de Consultas para a coleção de artigos obtidos no site Carta Capital

### Importando-se os pacotes necessários para a atividade

In [1]:
import pandas as pd
import re
import operator
import time
import sys
from math import log
from collections import Counter
from unicodedata import normalize

### Obtendo-se coleção do site Carta Capital

In [2]:
data_frame = pd.read_csv("results.csv")
documentos = data_frame["text"].tolist()

### Gerando índices invertidos (feito no último lab)

#### Algoritmo "parse" para obter os tokens de um determinado documento

In [102]:
def is_pontuacao(token):
    '''
        Retorna-se um booleano indicando se o token passado por parâmetro é uma pontuação.
    '''
    lista_pontuacoes = [",", ".", "!", "?", ":", "/", "#", "*", "(", ")", ";", ""]
    return token.strip() in lista_pontuacoes

def remove_acentuacoes(token):
    '''
        Remove as acentuações de um token.
    '''
    token_unicode = unicode(token, "utf-8")
    return normalize('NFKD', token_unicode).encode('ASCII','ignore')

def parse(texto):
    '''
        Transforma o texto em uma lista de tokens, eliminando espaços vazios, acentuações e pontuações, e
        tranforma todas as palavras em letras minúsculas.
    '''
    texto = re.sub(r'[,.!?:*();]', ' ', str(texto))
    texto = re.sub(r'\n', ' ', texto)
    lista_texto = texto.split(' ')

    tokens = []
    for palavra in lista_texto:
        token = palavra.lower()
        token = token.strip()
        token = remove_acentuacoes(token)

        if(not is_pontuacao(token)):
            tokens.append(token)
    
    return tokens

#### Algoritmo para obtenção dos índices invertidos

In [103]:
stop_words = ["para", "como", "esta", "mais", "pelo", "pela", "sobre", "isso", "essa", "esse", "essa", "aquele", "aquilo",
             "seus", "suas", "nosso", "nossos", "nossas", "entao", "assim", "ainda", "porem"]

def get_indices_invertidos(documentos):
    '''
        Obtém uma lista de índices invertidos a partir de uma lista de documentos.
    '''
    indices_invertidos = {}
    for i in range(len(documentos)):
        doc = parse(documentos[i])
        documento_counter = Counter(doc)
        
        for indice in documento_counter.keys():
            if(len(indice) > 3 and indice not in stop_words): # Evita stop words
                if(indice in indices_invertidos):
                    indices_invertidos[indice][i] = documento_counter[indice]
                else:
                    indices_invertidos[indice] = { i: documento_counter[indice] }
    
    return indices_invertidos

#### Obtendo-se os índices invertidos a partir dos documentos obtidos do Carta Capital

In [104]:
indices_invertidos = get_indices_invertidos(documentos)

### 1) Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

### Palavras escolhidas na questão 2.1 do  lab anterior

1. bolsonaro
2. brasil
3. casa
4. nome
5. vida

### Algoritmos para métricas

#### Mutual Information MIM

In [105]:
def mim(na, nb, nab):
    '''
        Calcula o Mutual Information (MIM)
    '''
    return nab / float(na*nb)

#### Expected Mutual Information (EMIM)

In [106]:
def emim(na, nb, nab, N):
    '''
        Calcula o Expected Mutual Information (EMIM)
    '''
    if nab == 0:
        return 0
    return nab * log(N * nab / float(na*nb))

#### Chi-Square (X²)

In [107]:
def chi_square(na, nb, nab, N):
    '''
        Calcula o Chi-Square (X²)
    '''    
    numerador = (nab - (na*nb)/N) ** 2
    denominador = float(na * nb)
    return numerador / denominador

#### Dice's Coefficient (Dice)

In [108]:
def dice(na, nb, nab):
    '''
        Calcula o Dice's Coefficient (Dice)
    '''
    return nab / float(na + nb)

### Calcular coeficientes

In [109]:
def getInterecaoListaPalavras(lista_ids_palavra_1, lista_ids_palavra_2):
    '''
        Obtém a interseção entre duas listas
    '''
    intersecao = []
    for id1 in lista_ids_palavra_1:
        for id2 in lista_ids_palavra_2:
            if id1 == id2:
                intersecao.append(id1)
    return intersecao

def getKey(item):
    '''
        Obtém a chave de cada item
    '''
    return item[1]

palavras_escolhidas = ["bolsonaro", "brasil", "casa", "nome", "vida"]

rankings = {"bolsonaro": {"mim": [], "emim": [], "x2": [], "dice": []},
            "brasil": {"mim": [], "emim": [], "x2": [], "dice": []},
            "casa": {"mim": [], "emim": [], "x2": [], "dice": []},
            "nome": {"mim": [], "emim": [], "x2": [], "dice": []},
            "vida": {"mim": [], "emim": [], "x2": [], "dice": []}}

for palavra in palavras_escolhidas:
    N = len(indices_invertidos.keys())
    na = len(indices_invertidos[palavra])
    
    for indice in indices_invertidos.keys():
        if(indice != palavra):
            nb = len(indices_invertidos[indice])
            nab = len(getInterecaoListaPalavras(indices_invertidos[palavra].keys(), indices_invertidos[indice].keys()))
            
            if(nab > 0): # Se não tem interseção, então são independentes e não entram nas contas
                calc_mim = mim(na, nb, nab)
                calc_emim = emim(na, nb, nab, N)
                calc_chi_square = chi_square(na, nb, nab, N)
                calc_dice = dice(na, nb, nab)
                
                rankings[palavra]["mim"].append((indice, calc_mim))
                rankings[palavra]["emim"].append((indice, calc_emim))
                rankings[palavra]["x2"].append((indice, calc_chi_square))
                rankings[palavra]["dice"].append((indice, calc_dice))
            
    rankings[palavra]["mim"] = sorted(rankings[palavra]["mim"], reverse=True, key=getKey)[:10]
    rankings[palavra]["emim"] = sorted(rankings[palavra]["emim"], reverse=True, key=getKey)[:10]
    rankings[palavra]["x2"] = sorted(rankings[palavra]["x2"], reverse=True, key=getKey)[:10]
    rankings[palavra]["dice"] = sorted(rankings[palavra]["dice"], reverse=True, key=getKey)[:10]

#### Tabela para as palavras mais associadas a bolsonaro

In [110]:
colunas = ["MIM", "EMIM", "X²", "Dice"]

palavras_associadas_mim_bolsonaro = [palavra for palavra, pontuacao in rankings["bolsonaro"]["mim"]]
palavras_associadas_emim_bolsonaro = [palavra for palavra, pontuacao in rankings["bolsonaro"]["emim"]]
palavras_associadas_x2_bolsonaro = [palavra for palavra, pontuacao in rankings["bolsonaro"]["x2"]]
palavras_associadas_dice_bolsonaro = [palavra for palavra, pontuacao in rankings["bolsonaro"]["dice"]]

tabela_palavras_associadas_bolsonaro = pd.DataFrame({"MIM": palavras_associadas_mim_bolsonaro, 
                                 "EMIM": palavras_associadas_emim_bolsonaro,
                                 "X²": palavras_associadas_x2_bolsonaro,
                                 "Dice": palavras_associadas_dice_bolsonaro})
tabela_palavras_associadas_bolsonaro = tabela_palavras_associadas_bolsonaro.reindex(columns=colunas)

tabela_palavras_associadas_bolsonaro

,MIM,EMIM,X²,Dice
0,sucateamento,presidente,jair,jair
1,bloqueou,tambem,presidente,presidente
2,scola,governo,brasil,brasil
3,capoeira,brasil,governo,governo
4,linterpol,jair,tambem,tambem
5,minimizaria,anos,deputado,contra
6,membros,disse,contra,mesmo
7,fuera,contra,campanha,disse
8,anulou,entre,disse,quando
9,guardados,mesmo,mesmo,anos


#### Tabela para as palavras mais associadas a brasil

In [111]:
colunas = ["MIM", "EMIM", "X²", "Dice"]

palavras_associadas_mim_brasil = [palavra for palavra, pontuacao in rankings["brasil"]["mim"]]
palavras_associadas_emim_brasil = [palavra for palavra, pontuacao in rankings["brasil"]["emim"]]
palavras_associadas_x2_brasil = [palavra for palavra, pontuacao in rankings["brasil"]["x2"]]
palavras_associadas_dice_brasil = [palavra for palavra, pontuacao in rankings["brasil"]["dice"]]

tabela_palavras_associadas_brasil = pd.DataFrame({"MIM": palavras_associadas_mim_brasil, 
                                 "EMIM": palavras_associadas_emim_brasil,
                                 "X²": palavras_associadas_x2_brasil,
                                 "Dice": palavras_associadas_dice_brasil})
tabela_palavras_associadas_brasil = tabela_palavras_associadas_brasil.reindex(columns=colunas)

tabela_palavras_associadas_brasil

,MIM,EMIM,X²,Dice
0,sucateamento,tambem,tambem,tambem
1,bloqueou,governo,pais,pais
2,francesco,pais,bolsonaro,bolsonaro
3,prestadoras,presidente,governo,governo
4,scola,anos,presidente,presidente
5,atingia,segundo,anos,anos
6,capoeira,entre,entre,entre
7,entrevistadas,bolsonaro,segundo,segundo
8,linterpol,foram,politica,mesmo
9,minimizaria,disse,quando,quando


#### Tabela para as palavras mais associadas a casa

In [112]:
colunas = ["MIM", "EMIM", "X²", "Dice"]

palavras_associadas_mim_casa = [palavra for palavra, pontuacao in rankings["casa"]["mim"]]
palavras_associadas_emim_casa = [palavra for palavra, pontuacao in rankings["casa"]["emim"]]
palavras_associadas_x2_casa = [palavra for palavra, pontuacao in rankings["casa"]["x2"]]
palavras_associadas_dice_casa = [palavra for palavra, pontuacao in rankings["casa"]["dice"]]

tabela_palavras_associadas_casa = pd.DataFrame({"MIM": palavras_associadas_mim_casa, 
                                 "EMIM": palavras_associadas_emim_casa,
                                 "X²": palavras_associadas_x2_casa,
                                 "Dice": palavras_associadas_dice_casa})
tabela_palavras_associadas_casa = tabela_palavras_associadas_casa.reindex(columns=colunas)

tabela_palavras_associadas_casa

,MIM,EMIM,X²,Dice
0,minimizaria,presidente,branca,branca
1,membros,tambem,presidente,outra
2,anulou,governo,lorenzoni,deputado
3,guardador,segundo,outra,tinha
4,derrotado,quando,quando,presidente
5,dialoga,entre,deputado,quem
6,promovemos,desde,desde,camara
7,declaracao colocando,mesmo,mesmo,quando
8,planices,disse,quem,fazer
9,chamava,depois,depois,desde


#### Tabela para as palavras mais associadas a nome

In [113]:
colunas = ["MIM", "EMIM", "X²", "Dice"]

palavras_associadas_mim_nome = [palavra for palavra, pontuacao in rankings["nome"]["mim"]]
palavras_associadas_emim_nome = [palavra for palavra, pontuacao in rankings["nome"]["emim"]]
palavras_associadas_x2_nome = [palavra for palavra, pontuacao in rankings["nome"]["x2"]]
palavras_associadas_dice_nome = [palavra for palavra, pontuacao in rankings["nome"]["dice"]]

tabela_palavras_associadas_nome = pd.DataFrame({"MIM": palavras_associadas_mim_nome, 
                                 "EMIM": palavras_associadas_emim_nome,
                                 "X²": palavras_associadas_x2_nome,
                                 "Dice": palavras_associadas_dice_nome})
tabela_palavras_associadas_nome = tabela_palavras_associadas_nome.reindex(columns=colunas)

tabela_palavras_associadas_nome

,MIM,EMIM,X²,Dice
0,prestadoras,tambem,tambem,epoca
1,lenco,brasil,brasil,movimento
2,capoeira,disse,blog,conta
3,membros,governo,disse,numero
4,guardador,presidente,bolsonaro,republica
5,planices,bolsonaro,apos,nada
6,ex-militares,apos,conta,jornal
7,surgiam,afirmou,afirmou,gabinete
8,rompem,anos,epoca,apos
9,boleto,foram,movimento,dezembro


#### Tabela para as palavras mais associadas a nome

In [114]:
colunas = ["MIM", "EMIM", "X²", "Dice"]

palavras_associadas_mim_vida = [palavra for palavra, pontuacao in rankings["vida"]["mim"]]
palavras_associadas_emim_vida = [palavra for palavra, pontuacao in rankings["vida"]["emim"]]
palavras_associadas_x2_vida = [palavra for palavra, pontuacao in rankings["vida"]["x2"]]
palavras_associadas_dice_vida = [palavra for palavra, pontuacao in rankings["vida"]["dice"]]

tabela_palavras_associadas_vida = pd.DataFrame({"MIM": palavras_associadas_mim_vida, 
                                 "EMIM": palavras_associadas_emim_vida,
                                 "X²": palavras_associadas_x2_vida,
                                 "Dice": palavras_associadas_dice_vida})
tabela_palavras_associadas_vida = tabela_palavras_associadas_vida.reindex(columns=colunas)

tabela_palavras_associadas_vida

,MIM,EMIM,X²,Dice
0,sucateamento,tambem,tambem,quando
1,lenco,quando,quando,onde
2,scola,anos,todos,todos
3,entrevistadas,presidente,mesmo,tempo
4,linterpol,brasil,onde,pode
5,minimizaria,mesmo,pode,quem
6,membros,foram,quem,mesmo
7,guardador,governo,anos,todo
8,guardados,segundo,tempo,enquanto
9,apelidos,pode,minha,agora


### Qual métrica você acha que obteve os melhores resultados? Por que?

O X² foi a métrica que apresentou os melhores resultados, principalmente nas palavras - bolsonaro e brasil -, pois as mesmas são mais específicas que as palavras: casa, nome e vida. Note que quando pesquisamos por bolsonaro, esperamos termos como jair, presidente, governo, política, deputado e camapanha. Assim como em brasil, espera-se termos como país, governo, presidente.
Vale-se salientar que esses termos vem de um site que debate muito sobre política, e os termos foram obtidos de artigos de 2018 até o início de 2019, fatos esses que propiciam uma grande discussão sobre política e sobre a presidência em questão.
Neste tocante, acredito que o X² foi mais bem sucedido em seus resultados.

### 2) De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

#### Algoritmo, implementado conforme é apresentado no livro texto, que executa a estratégia "documento por vez"¶

In [130]:
def consulta_conjuntiva_documento_por_vez(consulta, k, indices_invertidos):
    L = []
    R = []

    for termo in consulta:
        li = indices_invertidos.get(termo)
        if li is not None:
            L.append(li)

    primeiro_em_indices_invertidos = L[0]

    for doc in primeiro_em_indices_invertidos:
        score = 0
        d = -1

        for li in L:
            if doc in li:
                d = doc
                score += li[doc]
            else:
                d = -1
                break

        if d > -1:
            R.append((score, doc))

    R.sort(key = operator.itemgetter(0), reverse=True)

    return R[:k]

#### Levando-se em consideração o termo bolsonaro, e sabendo que para o X², o termo mais associado é Jair. No entanto, outros termos como presidente, brasil, camapnha e governo, estão bem associados. Logo, realizamos a consulta para o documento por vez com esses termos mais bem associados, a seguir.

#### Top 3 documentos

In [146]:
documentos_top_3 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "jair"], 3, indices_invertidos)
print documentos_top_3

documentos_top_3 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "presidente"], 3, indices_invertidos)
print documentos_top_3

documentos_top_3 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "brasil"], 3, indices_invertidos)
print documentos_top_3

documentos_top_3 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "campanha"], 3, indices_invertidos)
print documentos_top_3

documentos_top_3 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "governo"], 3, indices_invertidos)
print documentos_top_3

[(26, 425), (24, 31), (18, 168)]
[(32, 425), (26, 31), (26, 338)]
[(33, 425), (24, 374), (23, 104)]
[(26, 425), (22, 338), (21, 348)]
[(24, 425), (19, 338), (18, 31)]


#### Top 5 documentos

In [145]:
documentos_top_5 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "jair"], 5, indices_invertidos)
print documentos_top_5

documentos_top_5 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "presidente"], 5, indices_invertidos)
print documentos_top_5

documentos_top_5 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "brasil"], 5, indices_invertidos)
print documentos_top_5

documentos_top_5 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "campanha"], 5, indices_invertidos)
print documentos_top_5

documentos_top_5 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "governo"], 5, indices_invertidos)
print documentos_top_5

[(26, 425), (24, 31), (18, 168), (18, 355), (15, 108)]
[(32, 425), (26, 31), (26, 338), (25, 3), (23, 12)]
[(33, 425), (24, 374), (23, 104), (23, 338), (22, 355)]
[(26, 425), (22, 338), (21, 348), (18, 12), (15, 168)]
[(36, 425), (26, 12), (24, 348), (24, 374), (22, 355)]


#### Top 10 documentos

In [151]:
documentos_top_10 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "jair"], 10, indices_invertidos)
print documentos_top_10
print sum([score for score, id in documentos_top_10])

documentos_top_10 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "presidente"], 10, indices_invertidos)
print documentos_top_10

documentos_top_10 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "brasil"], 10, indices_invertidos)
print documentos_top_10

documentos_top_10 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "campanha"], 10, indices_invertidos)
print documentos_top_10

documentos_top_10 = consulta_conjuntiva_documento_por_vez(["bolsonaro", "governo"], 10, indices_invertidos)
print documentos_top_10

[(26, 425), (24, 31), (18, 168), (18, 355), (15, 108), (15, 335), (15, 383), (14, 79), (13, 147), (13, 374)]
171
[(32, 425), (26, 31), (26, 338), (25, 3), (23, 12), (21, 348), (21, 373), (20, 108), (20, 355), (19, 383)]
[(33, 425), (24, 374), (23, 104), (23, 338), (22, 355), (20, 31), (19, 77), (18, 238), (18, 375), (18, 383)]
[(26, 425), (22, 338), (21, 348), (18, 12), (15, 168), (15, 375), (13, 374), (12, 147), (12, 238), (12, 335)]
[(36, 425), (26, 12), (24, 348), (24, 374), (22, 355), (21, 338), (19, 238), (17, 373), (17, 383), (16, 108)]


#### Análise

Observe que para o top 3 documentos, os termos "presidente", "brasil" e, principalmente, "governo" tem maior score que o termo "jair", o que contrasta com o ranking gerado do X² que indica que o mais associado é "jair". Esse contraste se mantém a medida que aumentamos o nosso top para 5 ou para 10. Observe que com 10, o termo "governo" tem um score bem mais acentuado nos documentos que o termo "jair", enquanto que no ranking, ele é apenas o quarto lugar. Além desse termo, os termos "presidente" e "brasil" também tem mais score que o termo "jair". Sendo assim, a qualidade das métricas mostram-se boas, mas ainda um pouco diferentes do realmente esperado para o ranking.